# 1. Introduction
NFL is one of the biggest sports in the USA and analytics have a great impact on this sport as a number of plays are picked each game by looking at numbers. This game is about scoring more points than the rival team, to do so you have to get the ball from your field to the rival's endzone so you have to try getting the best out of every chance to move the ball.
Of course there are different ways to carry the ball to the rival's endzone, you either do it by passing it in the air to a teammate or you can also carry the ball through the ground by running with it, this is something we'll look into it to see what is better.
In order to analyze the different stats that an NFL game gives us we'll be using the dataset [NFL Box Scores](https://www.kaggle.com/grayengineering425/nfl-box-scores), there are two datasheets in this link and we'll be using the *box_scores.csv* to get our data.

# 2. Explaining the game
Before we look into our data, why not get familiar with the game of football (or american football for us the europeans)?
There are two teams competing for victory in a 120 yards (109.7m) long field although it really is 100 yards (91.44m) as 20 yards are the endzones and no plays are seen there, it's just there for scoring.
Let's dive into a simple list of features of the game:
-   **Objective:** Get the ball to the rival's endzone
-   **Game Time:** There are 4 playing windows called quarters that are 15 minutes long each
-   **How to get to the endzone:** When you are on the attack there are 4 opportunities to travel 10yards with the ball (either by air or by ground), this opportunities are called downs.
-   **Scoring:** You get 6 points for getting the ball to the endzone (**TOUCHDOWN!**) or 3 points if you score a field goal kick (with the feet, like us europeans!!). You also can kick an extra point or go for 2 points when a touchdown is scored, so the possiblities are: 3,6,7 or 8 points in an attacking play. When you are defending you can score by intercepting a pass or taking the ball away from your rival and getting it to the endzone, that would be anooother **TOUCHDOWN** but if you tackle an attacker on their endzone you will score 2 points.
-   **Drawing:** If no team is winning by the end of the 4th quarter we go into overtime: another 15 minute quarter where the first team to score a touchdown gets the win, or if no touchdown is scored the win will go to the team that has a greater score by the end of the overtime period. But what if at the end of this 15 extra minutes we still see a draw? The game ends with a draw at pro level while in college you could get a 10 quarter game because someone has to win.

<div style="text-align:center" markdown="1">
   <img src="https://i.giphy.com/media/xUPGcB5L0TIfLrDH8c/giphy.webp">
</div>

# 3. Dependencies
Let's talk about the libraries that you will need to run this notebook.
-  **Pandas**: all data and matrix operations, without we can't do nothing.



In [ ]:
import pandas as pd
import seaborn as sns